---
title: Functional Indexes in Postgres
description: Debugging high CPU load in postgres and compiling orm queries
date: 2-25-2023
categories:
  - flask
  - debugging
  - postgres
  - sql
image: ''
format:
    html:
        toc: true
        toc-location: left
        number-sections: true
---

I was recently diagnosing a high number of 500 errors in a web application built with [Flask](https://flask.palletsprojects.com) and found that the frequency of 5XX response codes correlated with a high CPU% usage on the postgres database that the flask app connected to.

![Correlation of high DB CPU usage with 5XX http response codes](dbcpu-httperr.png)

Digging into the logs of the flask app we saw a bunch of errors from [psycopg2](): a python package for talking to the postgres database:

```
sqlalchemy.exc.OperationalError: (psycopg2.OperationalError) SSL SYSCALL error: EOF detected
[
    <SELECT user.id, user.name...
    REST OF SQL QUERY HERE
    ...
    >
]

```

So naturally, I first checked if a `SSL SYSCALL error` was a common enough issue to show up on search and found [some](https://stackoverflow.com/questions/24130305/postgres-ssl-syscall-error-eof-detected-with-python-and-psycopg) [clues](https://dba.stackexchange.com/questions/315219/postgresql-foreign-data-wrappers-simultaneous-queries-wont-finish) as to what might be happening here. Other developers had run into this error and it ranged from seeing it during [long running/slow queries](https://stackoverflow.com/a/63130830/11538979), [low resources](https://stackoverflow.com/a/36086325/11538979) or [locked tables](https://stackoverflow.com/a/47874902/11538979).

This matched the observation of high DB CPU% utilisation that we saw on our app so the next step was figuring out what was causing this.

# Finding the problematic query

I was fairly confident that we had plenty of resources to handle "normal" load on our database since you can see the spikes at 8-9am were much higher than usual load - so there must've been something strange happening in the morning that doesn't happen during the rest of the day.

It wasn't unusual to see high traffic in the morning when users are waking up and logging into the website as one of their first activities of the day - but logging in didn't require use of a particularly intensive query - especially compared to the "heavier" analytical queries we had for complex analyses of large datasets. Regardless, it was still worth looking into this even just to rule out this possibility, especially given that the error did show a query that looked like it was selecting from the users table - suspiciously similar to a query you'd run while logging a user in! 

## Digging into Flask-security
One of the great things about Flask is its ecosystem of extensions that allow you to quickly integrate common features into your application such as [Flask-WTF](https://pypi.org/project/Flask-WTF/) for building forms, [Flask-SQLAlchemy](https://pypi.python.org/pypi/flask-sqlalchemy/) for integrating an ORM for your database integration and [Flask-Security](https://flask-security-too.readthedocs.io/en/stable/) which allows you to integrate common security mechanisms such as various types of authentication, user registration and sign in.

In this case, the Flask-Security extension (which itself is a combination of Flask extensions) manages the login of users to the application.

The extension implements a `/login` endpoint which renders a login form that runs a query against a datastore (in this case a postgres db) when submitted by the user. The login form class inherits from a Flask-WTF `BaseForm` class and implements a `validate` method to login the user:

```python
# https://github.com/mattupstate/flask-security/blob/master/flask_security/forms.py#L206
class LoginForm(Form, NextFormMixin):
    """The default login form"""

    email = StringField(get_form_field_label('email'),
                        validators=[Required(message='EMAIL_NOT_PROVIDED')])
    password = PasswordField(get_form_field_label('password'),
                             validators=[password_required])
    remember = BooleanField(get_form_field_label('remember_me'))
    submit = SubmitField(get_form_field_label('login'))

    def __init__(self, *args, **kwargs):
       ...

    def validate(self):
        ...
        self.user = _datastore.get_user(self.email.data)
        ...
```

The datastore is an instance of a `SQLAlchemyUserDataStore` which is responsible for running the query:

```python
# https://github.com/mattupstate/flask-security/blob/674b18103fa8734aca71bbd084ea01e3709817ef/flask_security/datastore.py#L227
class SQLAlchemyUserDatastore(SQLAlchemyDatastore, UserDatastore):
    """A SQLAlchemy datastore implementation for Flask-Security that assumes the
    use of the Flask-SQLAlchemy extension.
    """
    def __init__(self, db, user_model, role_model):
        SQLAlchemyDatastore.__init__(self, db)
        UserDatastore.__init__(self, user_model, role_model)

    def get_user(self, identifier):
        from sqlalchemy import func as alchemyFn
        if self._is_numeric(identifier):
            return self.user_model.query.get(identifier)
        for attr in get_identity_attributes():
            query = alchemyFn.lower(getattr(self.user_model, attr)) \
                == alchemyFn.lower(identifier)
            rv = self.user_model.query.filter(query).first()
            if rv is not None:
                return rv

```

The user model that was passed in to the constructor to instantiate this datastore during the flask application setup:

```python
class User(db.Model, UserMixin):
    id = db.Column(db.Integer, primary_key=True)
    email = db.Column(db.String(255), unique=True)
    password = db.Column(db.String(255))
    active = db.Column(db.Boolean())
    confirmed_at = db.Column(db.DateTime())
    roles = db.relationship(
        "Role",
        secondary=roles_users,
        backref=db.backref("users", lazy="dynamic"),
    )

```

In order of execution:

1. requests received for a protected page 
2. redirect to the login page with a `LoginForm`
3. user enters credentials and the form is submitted
4. run `.validate()` on the `LoginForm`
5. run `_datastore.get_user()` to fetch the user model
6. `SQLAlchemyUserDatastore` runs a query against the database

## Compiling ORM Queries
At this point we still don't have a SQL query that we can analyse because in step 6 the query is abstracted away through an ORM: `SQLALchemy` creates the query for us and maps it to an `User` model object:

```python
# https://github.com/mattupstate/flask-security/blob/674b18103fa8734aca71bbd084ea01e3709817ef/flask_security/datastore.py#L245
# in this case attr = email
query = alchemyFn.lower(getattr(self.user_model, attr)) \
    == alchemyFn.lower(identifier)
rv = self.user_model.query.filter(query).first()
```
Luckily we can stringify sqlachemy queries to generate the corresponding SQL query:

```python
print(self.user_model.query.filter(query))
```
```SQL
SELECT 
    "user".id AS user_id, 
    "user".email AS user_email, 
    "user".password AS user_password, 
    "user".active AS user_active,
     "user".confirmed_at AS user_confirmed_at 
FROM 
    "user" 
WHERE 
    lower("user".email) = lower(%(lower_1)s)
```

# Analysing the query execution plan
Now that we have a concrete query to work with, we can learn a bit about the execution plan that the PostgresSQL planner generates to see if we can spot anything unusual. 

```SQL
EXPLAIN ANALYZE 
SELECT 
    "user".id AS user_id, 
    "user".email AS user_email, 
    "user".password AS user_password, 
    "user".active AS user_active,
     "user".confirmed_at AS user_confirmed_at 
FROM 
    "user" 
WHERE 
    lower("user".email) = lower('test@email.com')

```
 |QUERY PLAN|
|----------|
|Seq Scan on "user"  (cost=0.00..2226.01 rows=500 width=32) (actual time=22.078..22.079 rows=0 loops=1)|
|  Filter: (lower((email)::text) = 'test@email.com'::text)|
|  Rows Removed by Filter: 100001|
|Planning Time: 0.106 ms|
|Execution Time: 22.093 ms|

The first thing that sticks out here is that we are using a sequential scan so this will not scale well as the number of users grows larger - since all the rows will be scanned sequentially (as the name implies). I assumed the query planner would pick index scan instead since we have a unique constraint in our model for user emails which created an index in our database: 


```SQL
SELECT 
    indexname, 
    indexdef 
FROM 
    pg_indexes 
WHERE 
    schemaname = 'public' AND
    tablename = 'user'
```
|indexname|indexdef|
|---------|--------|
|user_pkey|CREATE UNIQUE INDEX user_pkey ON public."user" USING btree (id)|
|user_email_key|CREATE UNIQUE INDEX user_email_key ON public."user" USING btree (email)|


However if we look closely at our query we can see that we use the `lower()` function on our email before running the query so it can't use the `user_email_key` index:

```SQL
...
WHERE lower("user".email) = lower("test@email.com")
...
```

We can test this out by removing the lower function and check if it uses the index:

```SQL
EXPLAIN ANALYZE 
SELECT 
    "user".id AS user_id, 
    "user".email AS user_email, 
    "user".password AS user_password, 
    "user".active AS user_active,
     "user".confirmed_at AS user_confirmed_at 
FROM 
    "user" 
WHERE 
    "user".email = 'test@email.com'
```

Bingo! As expected we use the index scan here so the `lower()` was indeed causing a problem:

|QUERY PLAN|
|----------|
|Index Scan using user_email_key on "user"  (cost=0.42..8.44 rows=1 width=32) (actual time=0.025..0.026 rows=0 loops=1)|
|  Index Cond: ((email)::text = 'test@email.com'::text)|
|Planning Time: 0.161 ms|
|Execution Time: 0.053 ms|

# Functional indexes
Now that we have found the cause, we can speed up this query with an index to solve this in a few ways:
1. change the query so it is case sensitive (i.e. remove the lower call) and ensure all email entries in the database are in lower case so we can use the existing `user_email_key` index
2. add a functional index

The first option would require us to change all exisiting emails in the database to lower case and add some validation or preprocessing of the email string during registration to ensure only lowercase emails are added which is more invasive than the second option.

Postgres supports the use of [functional indexes](https://www.postgresql.org/docs/7.3/indexes-functional.html) so we can define an index on the result of a function applied to a column:

```SQL
CREATE INDEX user_email_lower ON public."user" using btree (lower(email))

```

We can then run the case insensitive query again with the `lower()` call: we can see the new `user_email_lower` index is used!

|QUERY PLAN|
|----------|
|Index Scan using user_email_lower on "user"  (cost=0.42..8.44 rows=1 width=30) (actual time=0.033..0.034 rows=0 loops=1)|
|  Index Cond: (lower((email)::text) = 'test@email.com'::text)|
|Planning Time: 1.517 ms|
|Execution Time: 0.215 ms|

The total cost is reduced from 2226.01 to 8.44: a 263% increase! *(The first number is the startup cost of the query, this is expected to be lower for sequential scan since it can just start reading from the first row whereas for index scan it will have to read from the index before it can start reading the first row.)

# Reproducing the problem locally

So we've figured out the root cause of the problem and now can be quite confident that the issue is solved. However the real issue facing users wasn't necessarily a slow query - it was a slow/unresponsive website in the morning when they were trying to login.

To test the solution end to end (before deploying to staging/test environments) we can reproduce this scenario with a minimal example of a flask-security app backed by a postgres database. You can find the full demo example in the [repo here]() which lets you spin this up with a single `docker compose` command.

## Load testing with Locust